In [74]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 15 21:13:40 2017

@author: MKD
"""
import pandas as pd
import time
import re

def findMaxloca(lists):
    key = [i for i in range(len(lists))]
    dic = dict(zip(key,lists))
    dicc = {}
    for k,v in dic.items():
        dicc.setdefault(v,[]).append(k)
    return dicc[max(lists)]


def loadData(dfpath,respath):
    df = pd.read_csv(dfpath)
    res = pd.read_csv(respath)
    df_A = df[(df['target'] == 'A')]
    len_A = len(df_A)
    len_B = 100000 - len_A
    return res,df,df_A,len_A,len_B
    
def classRules(res,rules):
    starttime = time.time()
    new_rules = []
    rev_dic = {}
    reExp = r'X(.*?)=(.*?)[,}]'
    for rule in rules:
        invert = []
        invert = re.compile(reExp).findall(rule)
        tmp_1 = map(lambda x:int(x[0]) + int(x[1]),invert)
        tmp_2 = list(tmp_1)
        tmp_2.sort()
        tmp = "".join([str(i) for i in tmp_2])
        new_rules.append(tmp)
    dic_rul = dict(zip((res.ix[:,0]),new_rules))
    for k,v in dic_rul.items():
        rev_dic.setdefault(v,[]).append(k)
    endtime = time.time()
    totaltime = endtime - starttime
    return rev_dic,totaltime  

def findMaxInterestOfEachclass(res,rules,N,rev_dic,len_A,len_B):
    starttime = time.time()
    dic_sup = dict(zip((res.ix[:,0]),res.ix[:,2]))
    ruless = []
    interest = []
    for k,v in rev_dic.items():
        interOfEachRule = []
        for num in v:
            sup = dic_sup[num]
            inter = ((N*sup)*(N-N*sup))/(len_A*len_B)
            interOfEachRule.append(inter)
            break
        MaxInterOfEachRule = max(interOfEachRule)
        print(interOfEachRule)
        interest.append(MaxInterOfEachRule)
        lists = findMaxloca(interOfEachRule)
        #ruless.append(rules[v[interOfEachRule.index(MaxInterOfEachRule)]])
        #for i in lists:
            #print(i)
        ruless.append(rules[v[interOfEachRule.index(MaxInterOfEachRule)]])
    endtime = time.time()
    totaltime = endtime - starttime
    return interest,ruless,totaltime

if __name__ == '__main__':
    N = 10000
    dfpath = 'df.csv'
    respath = 'res.csv'
    #加载数据
    res,df,df_A,len_A,len_B = loadData(dfpath,respath)
    rules = res.ix[:,1]
    rev_dic,time1 = classRules(res,rules)
    interest,rules,time2 = findMaxInterestOfEachclass(res,rules,N,rev_dic,len_A,len_B)
    result_1 = pd.DataFrame.from_dict(rev_dic,orient='index')
    result_2 = pd.DataFrame({'interest':interest,'rules':rules},columns=['rules','interest'])

[0.00029717844855143178]
[0.0015267642710395453]
[0.00029770330267631161]
[0.00031763989242253348]
[0.00029560382247639376]
[0.00033284434412530567]
[0.00033389259788263345]
[0.00029875297907587197]
[0.00031449302903740036]
[0.0014897819217972916]
[0.00032078637360527572]
[0.00030977201733021782]
[0.00030452544017703918]
[0.00030872278683344675]
[0.00030872278683344675]
[0.00030924740739019879]
[0.00029297923335000221]
[0.00029140435247337031]
[0.00029665358380981876]
[0.00032183511573232522]
[0.00031449302903740036]
[0.00032131074997716701]
[0.0015037820803083018]
[0.00030662419843910762]
[0.00029087937094769348]
[0.00030452544017703918]
[0.00031816433249482324]
[0.00030662419843910762]
[0.0015057814234818144]
[0.0067261103762571205]
[0.00034489669839353666]
[0.00031868876195037993]
[0.00031239490777931663]
[0.00031921318078920348]
[0.00030505014566765596]
[0.00031344398964182471]
[0.00031344398964182471]
[0.00031921318078920348]
[0.00032550537874990624]
[0.0015187730135837476]
[0.000

In [68]:
o=[1,2,3,4,5,6,7,8,9]
p=findMaxloca(o)
p

[8]